In [1]:
import random
import time
from faker import Faker
from faker.providers import BaseProvider

# Adding a PizzaProvider with 3 methods:
#   * pizza_name to retrieve the name of the basic pizza,
#   * pizza_topping for additional toppings
#   * pizza_shop to retrieve one of the shops available
# https://github.com/Aiven-Labs/python-fake-data-producer-for-apache-kafka/tree/main


class PizzaProvider(BaseProvider):
    def pizza_name(self):
        valid_pizza_names = [
            "Margherita",
            "Marinara",
            "Diavola",
            "Mari & Monti",
            "Salami",
            "Peperoni",
        ]
        return random.choice(valid_pizza_names)

    def pizza_topping(self):
        available_pizza_toppings = [
            "🍅 tomato",
            "🧀 blue cheese",
            "🥚 egg",
            "🫑 green peppers",
            "🌶️ hot pepper",
            "🥓 bacon",
            "🫒 olives",
            "🧄 garlic",
            "🐟 tuna",
            "🧅 onion",
            "🍍 pineapple",
            "🍓 strawberry",
            "🍌 banana",
        ]
        return random.choice(available_pizza_toppings)

    def pizza_shop(self):
        pizza_shops = [
            "Marios Pizza",
            "Luigis Pizza",
            "Circular Pi Pizzeria",
            "Ill Make You a Pizza You Can" "t Refuse",
            "Mammamia Pizza",
            "Its-a me! Mario Pizza!",
        ]
        return random.choice(pizza_shops)

    def order_cost(self):
        return round(random.random() * 50, 2)

    # def produce_msg(
    #     self,
    #     FakerInstance,
    #     ordercount=1,
    #     max_pizzas_in_order=5,
    #     max_toppings_in_pizza=3,
    # ):
    #     shop = FakerInstance.pizza_shop()
    #     # Each Order can have 1-10 pizzas in it
    #     pizzas = []
    #     for pizza in range(random.randint(1, max_pizzas_in_order)):
    #         # Each Pizza can have 0-5 additional toppings on it
    #         toppings = []
    #         for topping in range(random.randint(0, max_toppings_in_pizza)):
    #             toppings.append(FakerInstance.pizza_topping())
    #         pizzas.append(
    #             {
    #                 "pizzaName": FakerInstance.pizza_name(),
    #                 "additionalToppings": toppings,
    #             }
    #         )
    #     # message composition
    #     message = {
    #         "id": ordercount,
    #         "shop": shop,
    #         "name": FakerInstance.unique.name(),
    #         "phoneNumber": FakerInstance.unique.phone_number(),
    #         "address": FakerInstance.address(),
    #         "pizzas": pizzas,
    #         "timestamp": int(time.time() * 1000),
    #     }
    #     key = {"shop": shop}
    #     return message, key

In [2]:
fake = Faker()
fake.add_provider(PizzaProvider)
Faker.seed(1234)

In [7]:
# creating function to generate the pizza Order
def produce_pizza_order (orderid = 1):
    shop = fake.pizza_shop()
    # Each Order can have 1-5 pizzas in it
    pizzas = []
    for pizza in range(random.randint(1, 5)):
        # Each Pizza can have 0-3 additional toppings on it
        toppings = []
        for topping in range(random.randint(0, 3)):
            toppings.append(fake.pizza_topping())
        pizzas.append({
            'pizzaName': fake.pizza_name(),
            'additionalToppings': toppings
        })
    # message composition
    message = {
        'id': orderid,
        'shop': shop,
        'name': fake.unique.name(),
        'phoneNumber': fake.unique.phone_number(),
        'address': fake.address(),
        'pizzas': pizzas,
        'cost': fake.order_cost(),
        "timestamp": int(time.time() * 1000),
    }
    key = {"shop": shop}
    return message, key

In [4]:
import json
from kafka import KafkaProducer

producer = KafkaProducer(
  bootstrap_servers=['localhost:9092'],
  value_serializer=lambda v: json.dumps(v).encode('ascii'),
  key_serializer=lambda v: json.dumps(v).encode('ascii')
)

In [10]:
topic_name = 'pizza-orders'
i = 0

while i <  5:
    message, key = produce_pizza_order(i)

    # print("Sending: {}".format(message))
    # sending the message to Kafka
    producer.send(topic_name,
                  key=key,
                  value=message)
    # 1-2 seconds of sleep time before the next message
    time.sleep(random.randint(1,2))

    # Force sending of all messages
    if (i % 100) == 0:
        producer.flush()
    i = i+1

producer.flush()

AssertionError: KafkaProducer already closed!

In [9]:
# Warning: Do not close the producer until you have finished all 3 notebooks!
producer.close()